In [1]:
import os
import sys
import pandas as pd
import numpy as np
import datetime as dt
import sqlalchemy as sa
import pyodbc
import warnings
import urllib

# Suppress warnings if necessary
warnings.filterwarnings('ignore')

# Ensure that the path to custom modules is correct and add it to sys.path
path_to_classes = "../2. Code/Classes"
sys.path.append(os.path.abspath(path_to_classes))

# Import custom modules
from Credentials_Loader import CredentialLoader
from Preprocessing_Application_N import DataPreprocessor

# Initialize the CredentialLoader and load credentials
credential_loader = CredentialLoader()
credentials = credential_loader.load_credentials()  # Ensure this method exists and properly reads from a .env or similar file

# Retrieve credentials
username = credentials['username']
password = credentials['password']
server = credentials['server']
database = credentials['reporting_db']
driver = '{ODBC Driver 17 for SQL Server}'  # Adjust based on your database and installed ODBC driver

# Print the database name to verify successful credential loading
print(database)

# Initialize the DataPreprocessor
processor = DataPreprocessor(server, database, username, password, driver)

# Print the current working directory for debugging
print("Current Working Directory:", os.getcwd())


# Set the path to the SQL file (ensure the path is correct relative to the current working directory)
path_to_sql = "../1. Data/Loan Portfolio Deli v2.sql"

# Fetch data using the DataPreprocessor
df = processor.fetch_data_from_sql(path_to_sql)


## almost never happnens 
df['CoappFlag'] = np.where((df['CoappFlag'] != 0) & (df['CoappFlag'] != 1) | (df['CoappFlag'].isna()), 0,  df['CoappFlag'])

main = df[df.CoappFlag == 0]

co = df[df.CoappFlag == 1]

main = main[~main.AccountNumber.isin(co.AccountNumber)]

df = pd.concat([main,co])

df = df[df.AccountStatus.isin(['OPEN','FROZEN','COLLECTION'])]




reporting-db
Current Working Directory: /Users/andreasnilsson/Library/CloudStorage/OneDrive-Nstart/Skrivbordet/Repository Homes/Behavior_Scorecard-DATA-223/2. Code


In [2]:
df[df.AccountNumber =='7700172']

,SnapshotDate,AccountNumber,IsOpen,AccountStatus,CurrentAmount,MOB,DisbursedDate,RemainingTenor,CoappFlag,Kronofogden,...,ExposureAtFirst60,ExposureAtFirst90,ExposureAtFirst120,ExposureAtFirstFrozen,forberanceDate,FBE_eftergift,ForberanceIn3Months,ForberanceIn6Months,ForberanceIn9Months,ForberanceIn12Months
11955,2024-05-31,7700172,1,OPEN,82826.57,29.0,2021-12-15,106,1.0,0,...,None,None,None,None,2024-01-24,0,0,1,1,1


In [3]:
FBE_ = df[df.FBE_eftergift == 1]
FBE_no = df[df.FBE_eftergift == 0]

# Step 2: Remove rows from FBE_no that have identical 'SnapshotDate' and 'AccountNumber' in FBE_
# Create a Boolean Series to identify rows to keep in FBE_no
mask = ~FBE_no[['SnapshotDate', 'AccountNumber']].apply(tuple, 1).isin(FBE_[['SnapshotDate', 'AccountNumber']].apply(tuple, 1))
# Apply the mask to filter FBE_no
FBE_no_filtered = FBE_no[mask]

# Step 3: Concatenate FBE_ and FBE_no_filtered into a new DataFrame
df = pd.concat([FBE_, FBE_no_filtered])

In [4]:
df

,SnapshotDate,AccountNumber,IsOpen,AccountStatus,CurrentAmount,MOB,DisbursedDate,RemainingTenor,CoappFlag,Kronofogden,...,ExposureAtFirst60,ExposureAtFirst90,ExposureAtFirst120,ExposureAtFirstFrozen,forberanceDate,FBE_eftergift,ForberanceIn3Months,ForberanceIn6Months,ForberanceIn9Months,ForberanceIn12Months
5356,2024-05-31,7130784,1,OPEN,137841.09,51.0,2020-02-05,120,0.0,0,...,None,None,None,None,2024-05-23,1,1,1,1,1
6328,2024-05-31,7168180,1,OPEN,307483.99,48.0,2020-05-29,192,0.0,0,...,None,None,None,None,2024-04-04,1,1,1,1,1
9188,2024-05-31,7252893,1,OPEN,50511.20,37.0,2021-04-21,82,0.0,0,...,None,None,None,None,2024-03-28,1,1,1,1,1
9393,2024-05-31,7269152,1,OPEN,270529.86,35.0,2021-06-09,204,0.0,0,...,None,None,None,None,2024-03-26,1,1,1,1,1
9732,2024-05-31,7268717,1,OPEN,58689.93,35.0,2021-06-08,131,0.0,0,...,None,None,None,None,2024-03-18,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21819,2024-05-31,9069311,1,OPEN,72430.00,0.0,2024-05-30,96,1.0,0,...,None,None,None,None,None,0,0,0,0,0
21820,2024-05-31,9069329,1,OPEN,490000.00,0.0,2024-05-30,144,1.0,0,...,None,None,None,None,None,0,0,0,0,0
21827,2024-05-31,9069808,1,OPEN,154495.00,0.0,2024-05-30,144,1.0,0,...,None,None,None,None,None,0,0,0,0,0
21829,2024-05-31,9070087,1,OPEN,50000.00,0.0,2024-05-30,60,1.0,0,...,None,None,None,None,None,0,0,0,0,0


In [5]:
path = "../1. Data"
os.chdir(path)

MacroInstrument = pd.read_excel('Macro_Instrument.xlsx')

#df.to_csv('BSC_Today.csv', index=False)

MacroInstrument = MacroInstrument[['Date','Instrument Rolling Mean']]
MacroInstrument['Instrument Rolling Mean'] = np.where(MacroInstrument['Instrument Rolling Mean'].notna(), MacroInstrument['Instrument Rolling Mean'],1 )    ## Will have 1 if NAN but this shall be updated each month 
MacroInstrument['SnapshotDate'] =  pd.to_datetime(MacroInstrument['Date'])
MacroInstrument

,Date,Instrument Rolling Mean,SnapshotDate
0,2019-01-31,1.000000,2019-01-31
1,2019-02-28,1.000000,2019-02-28
2,2019-03-31,1.000000,2019-03-31
3,2019-04-30,1.000000,2019-04-30
4,2019-05-31,0.988059,2019-05-31
...,...,...,...
65,2024-06-30,0.993619,2024-06-30
66,2024-07-31,0.992386,2024-07-31
67,2024-08-31,1.000000,2024-08-31
68,2024-09-30,1.000000,2024-09-30


In [6]:


#######################   CALCULATE BEHVAIOUR MODEL         ##################################
print('hej1')

coefficients = np.array([-0.44414603,  0.18778622 , 0.3539554 ,  0.70178643])
intercept = np.array([0])


# Sample DataFrame (Assuming you already have this in place)
pd_ = df.copy()


# Compute the Z values using your logistic regression model
pd_['Z'] = (pd_.CoappFlag * coefficients[0] +
            pd_.Ever30In6Months * coefficients[1] + 
            pd_.WorstDelinquency6M * coefficients[2] +
            pd_.CurrentDelinquencyStatus * coefficients[3] +

            intercept[0]) 

# Compute the original probabilities
pd_['P'] = pd_['Z'].apply(lambda x: 1 / (1 + np.exp(-x)))

# Coefficients and Intercept from the Calibration model
calibration_coef = 10.11569562
calibration_intercept = -8.18226514



print("Coefficient:", calibration_coef)
print("Intercept:", calibration_intercept)

# Using the original probabilities to calibrate them with the calibration model
pd_['Z_calibrated'] = pd_['P'].apply(lambda x: x * calibration_coef + calibration_intercept)

# Compute the calibrated probabilities
pd_['BehaviourModel'] = pd_['Z_calibrated'].apply(lambda x: 1 / (1 + np.exp(-x)))


pd_ = pd_[['AccountNumber','AccountStatus','SnapshotDate',	'MOB'	,'DisbursedDate',	'CurrentAmount','RemainingTenor','CoappFlag',
'Ever30In6Months',	'WorstDelinquency6M','CurrentDelinquencyStatus',
'WorstDelinquency12M','Ever30In12Months','Ever90In12Months'	,'P','BehaviourModel','Ever90',
'ForberanceIn6Months','ForberanceIn12Months','FBE_eftergift']] ## 'Score'	,'RiskClass', Had these before to controll results (OLD BEHAVIOUR MODEL)


pd_.loc[:, 'DisbursedDate'] = pd.to_datetime(pd_['DisbursedDate'])


BehaviourDone = pd_.copy()






hej1
Coefficient: 10.11569562
Intercept: -8.18226514


In [7]:

#######################         CALCULATE ADMISSION MODEL         ##################################

main_path = "../1. Data/MA Correct join - APL CRB-MLP Today.sql"
co_path = "../1. Data/CO Min score join - APL CBR MLP Today.sql"

preprocessor = DataPreprocessor(server, database, username, password,driver)
final_df = preprocessor.process_data(main_path, co_path)


pd_ = final_df[['SSN','PDScoreNew','UCScore','age' ,'Inquiries12M','UtilizationRatio','Amount','MaritalStatus','ReceivedDate','DisbursedDate','Applicationtype','Ever90',
                'Ever30',
                'AccountNumber','CapitalDeficit','PropertyVolume','PaymentRemarks','IndebtednessRatio','ApplicationScore', 'StartupFee','PaymentRemarksNo'] ]





# Assuming pd_ is your DataFrame and it's already defined

# Get the current date
now = dt.datetime.now()

# Get the first day of the current month
first_day_of_month = dt.datetime(now.year, now.month, 1)

# Ensure 'DisbursedDate' is in datetime format if it's not already
pd_.loc[:, 'DisbursedDate'] = pd.to_datetime(pd_['DisbursedDate'])


# Filter the DataFrame for rows where 'DisbursedDate' is less than the first day of the current month
pd_ = pd_[pd_['DisbursedDate'] < first_day_of_month]

# Print the maximum 'DisbursedDate' from the filtered DataFrame
print(pd_['DisbursedDate'].max())





# Coefficients and Intercept from the Logistic Regression model
coefficients = np.array([2.03675292e+00 ,-2.18071234e-02  ,3.39715771e-02, -2.12322589e-07])  
intercept = np.array([-0.13407141])

print("Coefficients:", coefficients)
print("Intercept:", intercept)


# Compute the Z values using your logistic regression model
pd_['Z'] = (pd_.UCScore * coefficients[0] +
            pd_.age * coefficients[1] + 
            pd_.Inquiries12M * coefficients[2] +
            pd_.PropertyVolume * coefficients[3] + 
            intercept[0]) 

# Compute the original probabilities
pd_['P'] = pd_['Z'].apply(lambda x: 1 / (1 + np.exp(-x)))

# Coefficients and Intercept from the Calibration model
calibration_coef = 3.7812065422080856
calibration_intercept = -4.336067082588543




# Using the original probabilities to calibrate them with the calibration model
pd_['Z_calibrated'] = pd_['P'].apply(lambda x: x * calibration_coef + calibration_intercept)

# Compute the calibrated probabilities
pd_['AdmissionModel'] = pd_['Z_calibrated'].apply(lambda x: 1 / (1 + np.exp(-x)))


AdmissionDone = pd_[['AccountNumber','PDScoreNew','UCScore','age','Inquiries12M','PropertyVolume','AdmissionModel','ApplicationScore']]
AdmissionDone['AccountNumber'] = AdmissionDone['AccountNumber'] #.astype(int)



2024-05-31 00:00:00
Coefficients: [ 2.03675292e+00 -2.18071234e-02  3.39715771e-02 -2.12322589e-07]
Intercept: [-0.13407141]


In [8]:

# Set option to display all columns
pd.set_option('display.max_columns', None)


pd_.tail(60)

,SSN,PDScoreNew,UCScore,age,Inquiries12M,UtilizationRatio,Amount,MaritalStatus,ReceivedDate,DisbursedDate,Applicationtype,Ever90,Ever30,AccountNumber,CapitalDeficit,PropertyVolume,PaymentRemarks,IndebtednessRatio,ApplicationScore,StartupFee,PaymentRemarksNo,Z,P,Z_calibrated,AdmissionModel
5208,Unknown,NaN,0.106,55,9.0,100.00,220000.0,MARRIED,2024-05-29,2024-05-30 00:00:00,2.0,0.0,NaN,9069063,11.0,0.0,0,0.2559,4.272936,NaN,0.0,-0.811823,0.307502,-3.173338,0.040181
5237,Unknown,NaN,0.012,28,-1.0,-100.00,415000.0,MARRIED,2024-05-29,2024-05-31 00:00:00,2.0,0.0,NaN,9071028,1.0,0.0,0,0.0000,0.945136,NaN,0.0,-0.754201,0.319907,-3.126434,0.042030
5211,Unknown,NaN,0.012,29,-1.0,-100.00,110000.0,PARTNER,2024-05-29,2024-05-29 00:00:00,1.0,0.0,NaN,9069121,12.0,513530.0,0,0.0000,1.150730,NaN,0.0,-0.885043,0.292134,-3.231448,0.037999
10618,8E13A0EF667FAFB22C081292427E22CE7679D3CDD0C5B2...,0.061464,0.100,24,7.0,93.10,10000.0,SINGLE,2024-05-29,2024-05-29 00:00:00,0.0,NaN,NaN,9068982,6.0,0.0,0,0.5240,5.406944,995.0,0.0,-0.215966,0.446217,-2.648827,0.066061
10619,651BF8283B7C5D02B8279CA1E3937972CE38FA1F996369...,0.036051,0.064,35,5.0,90.95,50000.0,PARTNER,2024-05-29,2024-05-29 00:00:00,0.0,NaN,NaN,9069188,-1.0,0.0,0,0.1698,1.150730,995.0,0.0,-0.597111,0.355005,-2.993720,0.047710
10621,22855A12FFCDDC82BABA4F2EA3AA0AD662B876D9F125A5...,0.098140,0.469,37,18.0,102.10,30000.0,SINGLE,2024-05-29,2024-05-31 00:00:00,0.0,NaN,NaN,9069220,32.0,0.0,0,1.3888,5.408990,995.0,0.0,0.625791,0.651534,-1.872481,0.133255
10622,364792E3FE064620E1F302EC855F30457A3376F69EF115...,0.027893,0.097,53,4.0,-100.00,10000.0,SINGLE,2024-05-29,2024-05-29 00:00:00,0.0,NaN,NaN,9069238,8.0,0.0,1,0.0985,0.945136,995.0,5.0,-0.956398,0.277600,-3.286404,0.036041
10623,7112804A975C10B0C8922546D655CE2E7A5642F57F66D4...,0.060283,0.176,33,27.0,95.52,15000.0,SINGLE,2024-05-29,2024-05-29 00:00:00,0.0,NaN,NaN,9069246,8.0,0.0,1,0.0602,4.272936,995.0,4.0,0.421995,0.603960,-2.052368,0.113813
10625,11B405BB5453BB9552635C362CB2D2B47210DBB385AB2F...,0.070772,0.137,24,8.0,94.83,35000.0,SINGLE,2024-05-29,2024-05-30 00:00:00,0.0,NaN,NaN,9069394,2.0,0.0,0,0.4173,5.406944,795.0,0.0,-0.106635,0.473367,-2.546170,0.072684
10626,BBDEC67107F071394DDBD136D8049B9CBD3B5DFF04C869...,0.044071,0.113,34,5.0,-100.00,100000.0,PARTNER,2024-05-29,2024-05-31 00:00:00,0.0,NaN,NaN,9069444,0.0,0.0,1,0.1895,3.054894,1995.0,3.0,-0.475503,0.383315,-2.886675,0.052816


In [ ]:

see['SICR'] = np.where((see.PD_Delta > 0.09) | (see['FBE'] != ''), 1, 0)


## Apply a lifetime factor, this is based from UCBLANCO VINTAGE ANALYSIS, in lower risk but still high 20 % increase and on the rest it will be 10 % increase

see['AdjustedBehaviourScore'] = np.where( (see['SICR'] == 1) &(see['AdjustedBehaviourScore'] < 0.50) , see.AdjustedBehaviourScore * 1.2 , 
                      np.where( (see['SICR'] == 1) &(see['AdjustedBehaviourScore'] >= 0.50) , see.AdjustedBehaviourScore * 1.1 ,   see.AdjustedBehaviourScore )) ## Adding LifeTime Convertion to Stage 2 


see['AdjustedBehaviourScore'] = np.where( see.AdjustedBehaviourScore > 1,1,see.AdjustedBehaviourScore)



see['Stageing'] = np.where(   (see['SICR'] == 0 ) 
                           
                           ,'Stage1',
                           np.where(   see['AdjustedBehaviourScore'] == 1.0 ,'Stage3','Stage2'))


see = see.drop_duplicates()


# Merge lek with MacroInstrument on 'SnapshotDate' in lek and 'Date' in MacroInstrument

see['SnapshotDate'] = pd.to_datetime(see['SnapshotDate'])


see = pd.merge(see, MacroInstrument, on='SnapshotDate', how='left')
see['AdjustedBehaviourScore'] =  see['BehaviourModel'] * see['Instrument Rolling Mean']
see['AdjustedBehaviourScore'] = np.where(  see['CurrentDelinquencyStatus'].isin([4,9]) ,1.0 , see['BehaviourModel'])


In [ ]:

#######################         CREATE SICR LOGIC         ##################################


AdmissionDone = pd_[['AccountNumber','PDScoreNew','UCScore','age','Inquiries12M','PropertyVolume','AdmissionModel','ApplicationScore']]
AdmissionDone['AccountNumber'] = AdmissionDone['AccountNumber'] # .astype(int)

together = pd.merge(BehaviourDone,AdmissionDone , on='AccountNumber', how='left')


## Only OPEN & FROZEN ACCOUNTS
lek = together[ (together.SnapshotDate >= min(together.SnapshotDate) )]



# Ensure DisbursedDate is a datetime object (if not already)
lek['DisbursedDate'] = pd.to_datetime(lek['DisbursedDate'])

# Convert the string to a datetime object
comparison_date = pd.to_datetime('2023-12-20')



lek['AppliedApplicationScore'] = np.where(
    (lek['DisbursedDate'] > comparison_date) &
    (np.round(lek['PDScoreNew'], 2) <= np.round(lek['AdmissionModel'], 2)) &
    (lek['PDScoreNew'].notna()), 
    lek['PDScoreNew'],  

    np.where(
        (lek['DisbursedDate'] > comparison_date) &
        
        lek['PDScoreNew'].isna(),  
        lek['AdmissionModel'],  

        np.where(
            (lek['DisbursedDate'] <= comparison_date), 
            lek['ApplicationScore'] / 100,  
            lek['AdmissionModel'] 
        )
    )
)

lek['AdjustedBehaviourScore'] = np.where(  lek['CurrentDelinquencyStatus'].isin([4,9]) ,1.0 , lek['BehaviourModel'])






see = lek.copy()


see['AppliedApplicationScore'] = np.where(  see.AppliedApplicationScore.isna()   ,0 , see.AppliedApplicationScore )


see['PD_Delta'] = see.AdjustedBehaviourScore - see.AppliedApplicationScore 


see = see[see.MOB.notna()]   ## take away accounts that was closed last month

see['PD_Delta'] = np.where(see['PD_Delta'].isna() , 0,see['PD_Delta'])

see = see.sort_values(by='PD_Delta')

# Assuming 'see' is your DataFrame
see['FBE'] = np.where(
    (see['ForberanceIn12Months'] == 1) & (see['CurrentDelinquencyStatus'] > 1) & (see['CurrentDelinquencyStatus'] < 4),
    'monitoring_previous_S3',
    np.where(
        see['FBE_eftergift'] == 1,
        'monitoring_paymentrelief',
        ''
    )
)



In [ ]:


see

,AccountNumber,AccountStatus,SnapshotDate,MOB,DisbursedDate,CurrentAmount,RemainingTenor,CoappFlag,Ever30In6Months,WorstDelinquency6M,CurrentDelinquencyStatus,WorstDelinquency12M,Ever30In12Months,Ever90In12Months,P,BehaviourModel,Ever90,ForberanceIn6Months,ForberanceIn12Months,FBE_eftergift,PDScoreNew,UCScore,age,Inquiries12M,PropertyVolume,AdmissionModel,ApplicationScore,AppliedApplicationScore,AdjustedBehaviourScore,PD_Delta,FBE,SICR,Stageing,Date,Instrument Rolling Mean
0,7100258,OPEN,2024-03-31,58.0,2019-05-27,78810.07,62,1.0,0,0,0,0,0,0,0.390753,0.014350,0,0,0,0,NaN,0.574,56.0,1.0,0.0,0.069846,19.834029,0.19834,0.014350,-0.183990,,0,Stage1,2024-03-31,0.997504
1,7152093,OPEN,2024-03-31,47.0,2020-04-06,124880.13,72,1.0,0,0,0,0,0,0,0.390753,0.014350,0,0,0,0,NaN,0.649,26.0,23.0,0.0,0.213903,19.834029,0.19834,0.014350,-0.183990,,0,Stage1,2024-03-31,0.997504
2,7748197,OPEN,2024-03-31,22.0,2022-05-09,219715.78,121,0.0,0,0,0,0,0,0,0.500000,0.042111,0,0,0,0,NaN,0.584,24.0,11.0,0.0,0.162004,19.834029,0.19834,0.042111,-0.156229,,0,Stage1,2024-03-31,0.997504
3,7681059,OPEN,2024-03-31,29.0,2021-10-15,204019.36,91,0.0,0,0,0,0,0,0,0.500000,0.042111,0,0,0,0,NaN,0.670,24.0,26.0,0.0,0.232348,19.834029,0.19834,0.042111,-0.156229,,0,Stage1,2024-03-31,0.997504
4,7249782,OPEN,2024-03-31,35.0,2021-04-12,88990.86,60,0.0,0,0,0,0,0,0,0.500000,0.042111,0,0,0,0,NaN,0.643,25.0,12.0,0.0,0.175990,19.834029,0.19834,0.042111,-0.156229,,0,Stage1,2024-03-31,0.997504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7356,5005111,FROZEN,2024-03-31,86.0,2017-01-31,203409.68,130,1.0,1,5,9,5,1,1,0.999602,0.873184,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,1.000000,1.000000,,1,Stage3,2024-03-31,0.997504
7357,5000294,FROZEN,2024-03-31,102.0,2015-09-25,338251.56,118,1.0,1,5,9,5,1,1,0.999602,0.873184,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,1.000000,1.000000,,1,Stage3,2024-03-31,0.997504
7358,5003488,FROZEN,2024-03-31,88.0,2016-11-25,94598.46,94,0.0,1,5,9,5,1,1,0.999745,0.873343,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,1.000000,1.000000,,1,Stage3,2024-03-31,0.997504
7359,5000419,FROZEN,2024-03-31,100.0,2015-11-05,143840.89,67,1.0,1,5,9,5,1,1,0.999602,0.873184,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,1.000000,1.000000,,1,Stage3,2024-03-31,0.997504


In [ ]:
output = see[['AccountNumber','SnapshotDate','AppliedApplicationScore','AdjustedBehaviourScore','PD_Delta'	,'FBE','SICR','Stageing']]

In [ ]:
output 

,AccountNumber,SnapshotDate,AppliedApplicationScore,AdjustedBehaviourScore,PD_Delta,FBE,SICR,Stageing
0,7100258,2024-03-31,0.19834,0.014350,-0.183990,,0,Stage1
1,7152093,2024-03-31,0.19834,0.014350,-0.183990,,0,Stage1
2,7748197,2024-03-31,0.19834,0.042111,-0.156229,,0,Stage1
3,7681059,2024-03-31,0.19834,0.042111,-0.156229,,0,Stage1
4,7249782,2024-03-31,0.19834,0.042111,-0.156229,,0,Stage1
...,...,...,...,...,...,...,...,...
7356,5005111,2024-03-31,0.00000,1.000000,1.000000,,1,Stage3
7357,5000294,2024-03-31,0.00000,1.000000,1.000000,,1,Stage3
7358,5003488,2024-03-31,0.00000,1.000000,1.000000,,1,Stage3
7359,5000419,2024-03-31,0.00000,1.000000,1.000000,,1,Stage3


In [ ]:
result = see.groupby('Stageing').agg({
    'AdjustedBehaviourScore': 'mean',
    'Score': 'mean'
}).reset_index()

# Rename columns to match the SQL output
result.columns = ['stageing', 'AdjustedBehaviourScore', 'Score']

# Display the result
print(result)

  stageing  AdjustedBehaviourScore       Score
0   Stage1                0.032779    3.239412
1   Stage2                0.455371   42.858891
2   Stage3                1.000000  100.000000


In [ ]:

# Filter out 'Stage3'
utan = see[see['Stageing'] != 'Stage3']

# Calculate the mean of 'AdjustedBehaviourScore' and 'Score' for the filtered data
result = utan[['AdjustedBehaviourScore', 'Score']].mean().reset_index()

# Convert the result into a DataFrame and rename columns to match SQL output
result = pd.DataFrame(result).T  # Transpose to flip rows and columns
result.columns = result.iloc[0]  # Set the first row as header
result = result[1:]  # Remove the first row after setting it as header

# Rename columns to match the SQL output
result.columns = ['AdjustedBehaviourScore', 'Score']

# Display the result
print(result)

  AdjustedBehaviourScore      Score
0               0.105963  10.100608


In [ ]:
see[see.AccountNumber =='7869431']

,AccountNumber,AccountStatus,SnapshotDate,MOB,DisbursedDate,CurrentAmount,RemainingTenor,CoappFlag,Ever30In6Months,WorstDelinquency6M,...,AdmissionModel,ApplicationScore,AppliedApplicationScore,AdjustedBehaviourScore,PD_Delta,FBE,SICR,Stageing,Date,Instrument Rolling Mean
8728,7869431,OPEN,2024-01-31,11.0,2023-02-21,364286.46,169,1.0,0,0,...,0.071613,1.15073,0.011507,0.01435,0.002843,monitoring_paymentrelief,1,Stage2,NaN,NaN
8800,7869431,OPEN,2023-12-31,10.0,2023-02-21,361845.29,170,1.0,0,0,...,0.071613,1.15073,0.011507,0.01435,0.002843,monitoring_paymentrelief,1,Stage2,NaN,NaN
9126,7869431,OPEN,2024-03-31,13.0,2023-02-21,93561.99,167,1.0,0,0,...,0.071613,1.15073,0.011507,0.01435,0.002843,,0,Stage1,NaN,NaN
9175,7869431,OPEN,2024-02-29,12.0,2023-02-21,93744.51,168,1.0,0,0,...,0.071613,1.15073,0.011507,0.01435,0.002843,,0,Stage1,NaN,NaN
9307,7869431,OPEN,2023-11-30,9.0,2023-02-21,362989.49,171,1.0,0,0,...,0.071613,1.15073,0.011507,0.01435,0.002843,monitoring_paymentrelief,1,Stage2,NaN,NaN


In [ ]:
see['FBE'].value_counts()

FBE
                            34577
monitoring_previous_S3        390
monitoring_paymentrelief      318
Name: count, dtype: int64

In [ ]:
see.AccountNumber.value_counts()

AccountNumber
7152093    5
7743578    5
7730427    5
5533351    5
7733090    5
          ..
9016007    1
7204878    1
9016478    1
9019506    1
9012790    1
Name: count, Length: 7947, dtype: int64

In [ ]:


see[see.AccountNumber == '7700172']

,AccountNumber,AccountStatus,SnapshotDate,MOB,DisbursedDate,CurrentAmount,RemainingTenor,CoappFlag,Ever30In6Months,WorstDelinquency6M,CurrentDelinquencyStatus,WorstDelinquency12M,Ever30In12Months,Ever90In12Months,Score,RiskClass,P,BehaviourModel,Ever90,ForberanceIn6Months,ForberanceIn12Months,FBE_eftergift,PDScoreNew,UCScore,age,Inquiries12M,PropertyVolume,AdmissionModel,ApplicationScore,AppliedApplicationScore,AdjustedBehaviourScore,PD_Delta,FBE,SICR,Stageing,Date,Instrument Rolling Mean
3216,7700172,OPEN,2024-03-31,27.0,2021-12-15,83568.69,108,1.0,0,0,0,0,0,0,1.877308,E3,0.390753,0.01435,0,1,1,1,NaN,0.001,52.0,2.0,1513100.0,0.025158,0.335849,0.003358,0.01435,0.010992,monitoring_paymentrelief,1,Stage2,2024-03-31,0.997504


In [ ]:
## 7810062 , 7733561


see[see.AccountNumber == '7810062'].sort_values(by='MOB')

,AccountNumber,AccountStatus,SnapshotDate,MOB,DisbursedDate,CurrentAmount,RemainingTenor,CoappFlag,Ever30In6Months,WorstDelinquency6M,CurrentDelinquencyStatus,WorstDelinquency12M,Ever30In12Months,Ever90In12Months,Score,RiskClass,P,BehaviourModel,Ever90,ForberanceIn6Months,ForberanceIn12Months,FBE_eftergift,PDScoreNew,UCScore,age,Inquiries12M,PropertyVolume,AdmissionModel,ApplicationScore,AppliedApplicationScore,AdjustedBehaviourScore,PD_Delta,FBE,SICR,Stageing,Date,Instrument Rolling Mean
47756,7810062,OPEN,2022-09-30,0.0,2022-09-22,337917.53,180,1.0,0,0,0,0,0,0,1.150730,N1,0.390753,0.014350,0,0,0,0,NaN,0.024,59.0,8.0,0.0,0.032554,1.15073,0.011507,0.014350,0.002843,,0,Stage1,NaN,NaN
47560,7810062,OPEN,2022-10-31,1.0,2022-09-22,337611.45,179,1.0,0,0,0,0,0,0,1.150730,N1,0.390753,0.014350,0,0,0,0,NaN,0.024,59.0,8.0,0.0,0.032554,1.15073,0.011507,0.014350,0.002843,,0,Stage1,NaN,NaN
47510,7810062,OPEN,2022-11-30,2.0,2022-09-22,337316.19,178,1.0,0,0,0,0,0,0,1.150730,N1,0.390753,0.014350,0,0,0,0,NaN,0.024,59.0,8.0,0.0,0.032554,1.15073,0.011507,0.014350,0.002843,,0,Stage1,NaN,NaN
47549,7810062,OPEN,2022-12-31,3.0,2022-09-22,241283.63,177,1.0,0,0,0,0,0,0,1.484099,E3,0.390753,0.014350,0,0,0,0,NaN,0.024,59.0,8.0,0.0,0.032554,1.15073,0.011507,0.014350,0.002843,,0,Stage1,NaN,NaN
49291,7810062,OPEN,2023-01-31,4.0,2022-09-22,241086.65,176,1.0,0,0,0,0,0,0,1.484099,E3,0.390753,0.014350,0,0,0,0,NaN,0.024,59.0,8.0,0.0,0.032554,1.15073,0.011507,0.014350,0.002843,,0,Stage1,NaN,NaN
148828,7810062,OPEN,2023-02-28,5.0,2022-09-22,241086.65,175,1.0,0,1,1,1,0,0,13.185339,E5,0.648305,0.164625,0,0,0,0,NaN,0.024,59.0,8.0,0.0,0.032554,1.15073,0.011507,0.164625,0.153118,,1,Stage2,NaN,NaN
161756,7810062,OPEN,2023-03-31,6.0,2022-09-22,241086.65,174,1.0,1,2,2,2,1,0,63.410000,E6,0.864728,0.637622,0,0,0,0,NaN,0.024,59.0,8.0,0.0,0.032554,1.15073,0.011507,0.637622,0.626114,,1,Stage2,NaN,NaN
161746,7810062,OPEN,2023-04-30,7.0,2022-09-22,240886.15,173,1.0,1,2,2,2,1,0,63.410000,E6,0.864728,0.637622,0,0,0,0,NaN,0.024,59.0,8.0,0.0,0.032554,1.15073,0.011507,0.637622,0.626114,,1,Stage2,NaN,NaN
168203,7810062,OPEN,2023-05-31,8.0,2022-09-22,240886.15,172,1.0,1,3,3,3,1,0,92.650000,E7,0.948381,0.803966,0,0,0,0,NaN,0.024,59.0,8.0,0.0,0.032554,1.15073,0.011507,0.803966,0.792459,,1,Stage2,NaN,NaN
190240,7810062,OPEN,2023-06-30,9.0,2022-09-22,240886.15,171,1.0,1,4,4,4,1,1,100.000000,E8,0.981414,0.851375,1,0,0,0,NaN,0.024,59.0,8.0,0.0,0.032554,1.15073,0.011507,1.000000,0.988493,,1,Stage3,NaN,NaN


In [ ]:
see[['']]

In [ ]:
set(see.ForbearanceName)

{'Capitalization',
 'Extension of maturity',
 None,
 'Payment Free Month',
 'Payment free month',
 'Permanent interest cut',
 'Temporary interest cut'}